![Image](../../resources/cropped-SummerWorkshop_Header.png)

<h1 align="center">Population Coding</h1> 
<h2 align="center"> SWDB 2024 - Day 2 - Afternoon Session </h2> 
<h3 align="center"> Tuesday, August 20, 2024</h3> 



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p> 

### Questions  

In the first workshop of today, we examined how sensory variables can be encoded in individual neurons' activity. 

We now turn our attention to the **coordinated activity of groups of neurons: population codes!**

Key questions:
* How do populations of neurons encode information about task-relevant sensory information? 
* How are these population codes modulated by task context or behavioral state? 
* What other types of thing are encoded in population activity?

![neural_coding.png](../../resources/neural_coding.png)

<br>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
### Extracellular Electrophysiology Data

The data from in vivo extracellular electrophysiology experiments are organized into *sessions*, where each session is a distinct continuous recording period. During a session we collect:

- spike times and characteristics (such as mean waveforms) from up to 6 neuropixels probes
- local field potentials
- behavioral data, such as running speed and eye position and lick times
- visual stimuli which were presented during the session
- cell-type specific optogenetic stimuli that were applied during the session

The AllenSDK contains code for accessing across-session (project-level) metadata as well as code for accessing detailed within-session data. The standard workflow is to use project-level tools, such as `EcephysProjectCache` to identify and access sessions of interest, then delve into those sessions' data using `EcephysSession`.

<div style="background: #E6E6FA; border-radius: 3px; padding: 10px;">
<p>

### Data access - loading an experiment of interest

In [ ]:
# We need to import these modules to get started
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import brain_observatory_utilities.datasets.behavior.data_formatting as behavior_utils

# seaborn makes pretty plots & sets font sizes nicely
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

# magic functions for jupyter notebook plotting
%matplotlib inline

#### Load the cache and get metadata tables

In [ ]:
import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # macOS 
    data_root = "/Volumes/Brain2024/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2024/"

In [ ]:
# import behavior projet cache class from SDK to be able to load the data
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache

cache = VisualBehaviorNeuropixelsProjectCache.from_local_cache(cache_dir=data_root, use_static_cache=True)

# if we needed to download the data we could have used the following line
# cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_root)  

In [ ]:
# get the metadata tables
units_table = cache.get_unit_table()

channels_table = cache.get_channel_table()

probes_table = cache.get_probe_table()

behavior_sessions_table = cache.get_behavior_session_table()

ecephys_sessions_table = cache.get_ecephys_session_table()

<br>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
### Grab data from a session

In [ ]:
session = cache.get_ecephys_session(ecephys_session_id=1065437523)

<br>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
The stimulus presentations table is a record of every stimulus we presented to the mouse over the course of this experiment. Let's take a look at this table. 
    
Here, we'll use an annotated version that includes some extra behavioral information.

In [ ]:
stimulus_presentations = behavior_utils.get_annotated_stimulus_presentations(session)
stimulus_presentations.head(-5)

#### It contains a great deal of information about the stimulus presentations! Let's look at all the columns:

In [ ]:
np.sort(stimulus_presentations.columns)

The experiment is divided into stimulus blocks. During each block a different set of stimuli are presented. A stimulus block can be active or passive. In active blocks, the mouse performs the change detection task introduced earlier. In passive blocks, there is no task.

The different types of stimuli are indexed by the 'stimulus_block' column. Notice that our annotated stimulus table only has block 0, in which natural images are shown. What are the other stimulus blocks?

<br>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">    

What are all the types of stimulus block that were presented in this session?

In [ ]:
all_stimulus_presentations = session.stimulus_presentations
all_stimulus_presentations.groupby('stimulus_block')[['stimulus_block', 
                                                'stimulus_name', 
                                                'active', 
                                                'duration', 
                                                'start_time']].head()

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
    
What stimuli were shown in stimulus block 0? (Remember: our "stimulus_presentations" table already contains only this block.)

### Moving forwards, we will only look at this stimulus block, 0, where the mouse is performing the change detection task.

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
    
How many stimulus presentations are there and how many image changes (in block 0)?

In [ ]:
change_stimulus_presentations = stimulus_presentations[stimulus_presentations['is_change']]


### The (annotated) stimulus presentation table also includes information about the mouse behavior.

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
    
How well does the mouse do the task? What are its hit and miss rates? 
    
(Note that the first few trials are auto-rewarded, and should not be counted.)

<div style="background: #E6E6FA; border-radius: 3px; padding: 10px;">
    
### Now let's get unit and channel data, sort the units by depth and filter for "good" units.

In [ ]:
### get unit and channel data, sort the units by depth and filter for "good" units
units = session.get_units() # contains information about spike waveforms, isolation quality
channels = session.get_channels() # contains information about anatomical location

unit_channels = units.merge(channels, left_on='peak_channel_id', right_index=True) # associate anatomical information with each unit

#first let's sort our units by depth and filter
unit_channels = unit_channels.sort_values('probe_vertical_position', ascending=False)

#now we'll filter them
good_unit_filter = ((unit_channels['snr']>1)&
                    (unit_channels['isi_violations']<1)&
                    (unit_channels['firing_rate']>0.1))

good_units = unit_channels.loc[good_unit_filter]

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
Which brain structures were recorded from in this session? How many units are present in each structure? (Hint: try the "value_counts" function.)

<br>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

### Let's look at the population activity in primary visual cortex

In [ ]:
area_of_interest = 'VISp'
area_units = good_units[good_units['structure_acronym'] == area_of_interest]
num_units = len(area_units)

### Let's start by looking at the neural activity! Does it reflect the image presentation?
### The session.spike_times object contains all spike times, in seconds, indexed by the unit ID. Let's take a look at this object.

In [ ]:
spike_times = session.spike_times
print(type(spike_times))
spike_times

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">

Get the array of spike times for unit 1068230173. How many times does this unit spike in the first minute of the experiment?

In [ ]:
unit_spike_times = 

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">

Plot a population spike raster spanning 1 second before to 1 second after a stimulus presentation. (Complete the two lines in the for loop.)

In [ ]:
### plot a single-trial raster, population PSTH, and representation matrix
pre_time = 1
post_time = 1

fig, ax = plt.subplots(1, 1)

presentation_idx = 1 # which trial to center the raster on
start_time = stimulus_presentations['start_time'][presentation_idx] # get spike times starting one second before this
end_time = stimulus_presentations['end_time'][presentation_idx] # 

unit_num = 0
for iu, unit in area_units.iterrows():
    unit_spike_times = spike_times[iu] # a numpy array
    
    unit_spike_times = 
    unit_num_spikes = 
    
    ax.plot(unit_spike_times - start_time, unit_num*np.ones(unit_num_spikes,), 'k|', markersize=5)
    unit_num += 1

ax.set_title('Single-trial raster')
ax.set_xlabel('Time relative to stimulus presentation (s)')
ax.set_ylabel('Unit')
ax.set_ylim((0, num_units+1))

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
    
Now let's compare to a change trial.

In [ ]:
change_idx = np.where(stimulus_presentations['is_change'].values)[0]
presentation_idx = change_idx[0]

start_time = stimulus_presentations['start_time'][presentation_idx]
end_time = stimulus_presentations['end_time'][presentation_idx]



### Now let's take a look at the trial-averaged responses to see how a neuron encodes the stimulus in its time-dependent firing rate

In [ ]:
#Convenience function to compute the PSTH
def makePSTH(spikes, startTimes, windowDur, binSize=0.001):
    bins = np.arange(0,windowDur+binSize,binSize)
    counts = np.zeros(bins.size-1)
    for i,start in enumerate(startTimes):
        startInd = np.searchsorted(spikes, start)
        endInd = np.searchsorted(spikes, start+windowDur)
        counts = counts + np.histogram(spikes[startInd:endInd]-start, bins)[0]
    
    counts = counts/startTimes.size
    return counts/binSize, bins

Let's start by plotting the response of unit 0 to one of the images.

In [ ]:
stimuli = stimulus_presentations['image_name'].unique()
stimulus = stimuli[0]

In [ ]:
presentations = stimulus_presentations[stimulus_presentations['image_name'] == stimulus]
num_presentations = len(presentations)

start_times = presentations['start_time'].values

In [ ]:
unit_ids = area_units.index
iu = unit_ids[5]
unit_spike_times = spike_times[iu]

In [ ]:
time_before_im = 1
duration = 2

unit_response, bins = makePSTH(unit_spike_times, 
                                  start_times - time_before_im, 
                                  duration, binSize=0.01)

fig, ax = plt.subplots(1, 1)
ax.plot(bins[:-1] - time_before_im, unit_response)
ax.set_xlabel('Time from change (s)')
ax.set_ylabel('Firing rate (Hz)')
ax.set_title('Peri-stimulus time histogram for {}'.format(stimulus))

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
Plot the PSTHs for every unit to that image.

### We can see the trial structure of the task reflected in the PSTH. Some units have very strong transient responses to the image presentation. Do these responses depend on the task structure (whether the image is a change or not)?

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
Plot the PSTHs for every unit to that image on change trials only. Are the same neurons the most responsive on change trials as on average?

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
Plot the PSTHs for every unit to another image on change trials. Do the same neurons have the strongest responses?

<div style="background: #E6E6FA; border-radius: 3px; padding: 10px;">
    
## Training a classifier on population spiking data

Now we'll look at how the population activity encodes the image change. To determine how well we can decode the image change from population activity, we will train a **classifier** on a matrix of firing rates. Whereas regression models try to predict continuous values from the input features, classification models try to predict *labels* (also known as classes) from the input features.

### Support Vector Machines

Let's start with a linear Support Vector Machine (SVM) classifier, which will try to draw linear boundaries between orientation conditions (the labels) in our high-dimensional firing rate space.

This cartoon shows how we would expect an SVM to behave on a dataset with two dimensions and three conditions:

![SVM illustration](../resources/svm-classifier.png)

SVM computes decision boundaries in feature space that can be used to classify different conditions. If a new data point appears, the SVM classifier will label it based on where it falls with respect to these boundaries.

To train an SVM, we need to import the following methods from `scikit-learn`:

In [ ]:
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

### First, we need to create a response matrix and vector of stimulus labels from the presentations that we'll decode.

In [ ]:
print('Total trials: {}'.format(len(stimulus_presentations)))
print('Change trials: {}'.format(np.sum(stimulus_presentations['is_change'].values)))

<br>
<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

The vast majority of stimulus presentations (~95%) are not a change. So a decoder could get 95% accuracy by predicting that there are no changes!

To avoid this, we will balance the trials and decode change vs pre-change.

In [ ]:
# add pre-change
stimulus_presentations['pre_change'] = stimulus_presentations['is_change'].shift(-1)
# broadcast hit/miss values from the change image to the pre-change
stimulus_presentations.loc[stimulus_presentations['pre_change'], 'hit'] = stimulus_presentations[stimulus_presentations['is_change']].hit.values
stimulus_presentations.loc[stimulus_presentations['pre_change'], 'miss'] =  stimulus_presentations[stimulus_presentations['is_change']].miss.values

# isolate the trials to use for decoding
decode_trial_ind = (stimulus_presentations['is_change'].values + stimulus_presentations['pre_change'].values)
stimulus_presentations = all_stimulus_presentations[decode_trial_ind]

In [ ]:
### check that the classes are balanced
print(len(stimulus_presentations))
print(np.sum(stimulus_presentations['is_change']))
print(np.sum(stimulus_presentations['pre_change']))

### Now let's make our matrix of responses and vector of labels.

In [ ]:
def make_response_array(spike_times, stimulus_presentations, units, window=.05):

    '''
    Create an array of spike counts x stimulus presentations, and a corresponding list of stimulus label
    spike_times: spike times 
    stimulus_presentation: stimulus presentation table
    units: units table containing only the units to get the responses of
    '''

    # sort spike times chronologically; necessary for the binary search later
    sorted_spikes = dict()
    for iu in units.index:
        # mergesort/timsort since most spike_times are already sorted
        sorted_spikes[iu] = np.sort(spike_times[iu], kind='mergesort')

    # create our own copy of stimulus presentations and sort by presentation start time chronologically
    # sortation of stimulus_presentations isn't necessary, but it speeds up the vectorized `searchsorted(...)`
    stimulus_presentations = stimulus_presentations.sort_values(by='start_time', kind='mergesort', inplace=False)

    # Calculate the duration of stimulus presentations, and drop NaN durations
    stimulus_presentations['duration'] = stimulus_presentations['end_time'] - stimulus_presentations['start_time']
    stimulus_presentations.dropna(subset='duration', inplace=True)
    
    # Warn if window size is too big
    if np.any(window > stimulus_presentations['duration']):
        print('Warning: window size longer than stimulus presentation')

    responses_by_unit = list()
    for iu in units.index:
        unit_spike_times = sorted_spikes[iu]

        # Determine the first and last spike time for each stimulus presentation
        start_is = np.searchsorted(unit_spike_times, stimulus_presentations['start_time'])
        end_is = np.searchsorted(unit_spike_times, stimulus_presentations['start_time']+window)

        # presentation_spike_times = unit_spike_times[start_i:end_i]

        # Calculate the response rate for each stimulus presentation
        responses_by_unit.append((end_is - start_is) / stimulus_presentations['duration'])

    # responses_by_unit has each row a unit, and each column a stimulus, flip so that rows are stimuli
    responses = np.transpose(responses_by_unit)

    # Extract the labels that match the responses from our sorted stimulus presentations table
    labels = np.array(stimulus_presentations['image_name'])
    
    return responses, labels

In [ ]:
responses, labels = make_response_array(spike_times, stimulus_presentations, area_units, window=.06)

### We will first select a random subset of trials for training the classifier:

In [ ]:
total_presentations = responses.shape[0]
num_train = int(total_presentations * 0.5) # Use 50% of trials for training
random_trial_order = np.random.permutation(responses.shape[0])
train_indices = random_trial_order[:num_train]

training_data = responses[train_indices]
training_labels = labels[train_indices]

### Next, we'll create the model and fit it to our training data:

In [ ]:
clf = svm.SVC()
clf.fit(responses[train_indices], labels[train_indices])

### Now that our model has been trained, we can ask it to classify unlabeled data (i.e., the sets of population firing rates that were not included in our original training set):

In [ ]:
test_indices = random_trial_order[num_train:]
test_data = responses[test_indices]
predicted_labels = clf.predict(responses[test_indices])

### We can compare the predicted labels to the actual labels in order to assess the classifier's performance. We'll assess accuracy as the fraction of correctly predicted test images. As a baseline, we'll also compute the accuracy of a uniform random prediction.

In [ ]:
conditions = np.unique(labels)

actual_labels = labels[test_indices]
accuracy = np.mean(actual_labels == predicted_labels)

print('Accurary: {}'.format(accuracy))
print('Chance level: {}'.format(1/len(conditions)))

### We see that we perform better than chance! We can get a better sense of classification performance by using leave-one-out cross-validation. The `scikit-learn.model_selection.LeaveOneOut` iterator will automatically cycle through trials, on each iteration using one trial as a test and the others to train the classifier. Note that the model is fit independently on each iteration.

In [ ]:
accuracies = []
confusions = []

for train_indices, test_indices in LeaveOneOut().split(responses):
    
    clf = svm.SVC()
    clf.fit(responses[train_indices], labels[train_indices])
    
    test_targets = labels[test_indices]
    test_predictions = clf.predict(responses[test_indices])
    
    accuracy = np.mean(test_targets == test_predictions)    
#     print(accuracy)
    
    accuracies.append(accuracy)
    confusions.append(confusion_matrix(y_true=test_targets, y_pred=test_predictions, labels=conditions, normalize='pred'))
    
print(f"\nmean accuracy: {np.mean(accuracies)}")
print(f"chance: {1/conditions.size}")

The leave-one-out cross-validation roughly agrees with our previous result. Do we do better on change or pre-change images? To assess this we'll look at the confusion matrix, which tells us how frequently each condition is predicted on change and pre-change presentations.

In [ ]:
def plot_confusion_matrix(confusions, conditions, title=None):
    
    mean_confusion = np.mean(confusions, axis=0)

    fig, ax = plt.subplots(1, 1)
    im = ax.imshow(mean_confusion, cmap='gray_r', clim=(0, 1))
    plt.colorbar(im, ax=ax, label='Fraction of classifications')
    
    ax.set_xticks(range(len(conditions)), conditions, rotation=0)
    ax.set_yticks(range(len(conditions)), conditions)

    ax.set_xlabel("Predicted label")
    ax.set_ylabel("Actual label")
    if title is None:
        ax.set_title('Confusion Matrix')
    elif type(title) is str:
        ax.set_title(title)
    
plot_confusion_matrix(confusions, conditions)

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
    
How does this V1-based decoder perform on hit trials vs miss trials?

<div style="background: #E6E6FA; border-radius: 3px; padding: 10px;">
    
## Exploring the time course of change-related information 
    
Next we'll examine the time course of information in our population! Or more specifically: how the length of the spike count window affects the decoding accuracy. Can we decode the stimulus perfectly if we integrate spikes for long enough?

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
    
First, let's try decoding with a longer response window of .2 seconds.

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
    
How long do we need to integrate spikes in order to saturate the decoding performance?
    
Here, we suggest using a relaxed, K-Fold cross-validation instead of LeaveOneOut for speed purposes. This divides the data set into K pieces, and iterates through them. On each of those K iterations, one piece is used as the "test" set and the others are used to train the decoder.

Fill in the code below:

In [ ]:
window_lengths = np.arange(.01, .2, .02)

In [ ]:
num_splits = 10
accuracies = np.zeros((len(window_lengths), num_splits))
accuracies_hit = np.zeros((len(window_lengths), num_splits))
accuracies_miss = np.zeros((len(window_lengths), num_splits))

for i, window in enumerate(window_lengths):
    print('{}/{}'.format(i, len(window_lengths)))

    k = 0
    for train_indices, test_indices in KFold(n_splits=num_splits, shuffle=True).split(responses):
        
        

How does this time course relate to the mouse's response times?

<div style="background: #E6E6FA; border-radius: 3px; padding: 10px;">

## Relationship between population size and decoding accuracy

Next we'll examine how the size of the simultaneously recorded population affects decoding accuracy. In any physiology experiment, we only have a very small window into the overall population response. For example, there are about 500,000 neurons in mouse V1, so in this case we are measuring around 0.02% of the firing rates in this region.

As the number of simultaneously recorded neurons increases, we expect that our ability to decode stimulus identity will improve. 

To start with, let's try decoding with a random sample of 10 neurons. Note: here we're using the responses from our longest window above. So, if we used the full population we would be able to decode almost perfectly.

In [ ]:
pop_size = 10

pop_idx = np.random.choice(range(num_units), size=pop_size)
responses_pop = responses[:, pop_idx]

accuracies = []
confusions = []

for train_indices, test_indices in KFold(n_splits=num_splits, shuffle=True).split(responses_pop):
    
    clf = svm.SVC()
    clf.fit(responses_pop[train_indices], labels[train_indices])

    test_targets = labels[test_indices]
    test_predictions = clf.predict(responses_pop[test_indices])

    accuracy = np.mean(test_targets == test_predictions)    

    accuracies.append(accuracy)
    confusions.append(confusion_matrix(y_true=test_targets, y_pred=test_predictions, labels=conditions, normalize='pred'))
    
print(f"\nmean accuracy: {np.mean(accuracies)}")
print(f"chance: {1/conditions.size}")

plot_confusion_matrix(confusions, conditions)

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">
    
Does the result depend on which 10 neurons we sampled? Let's try another random sample.

<div style="border-left: 3px solid #000; padding: 10px; padding-left: 10px; padding-bottom: 10px; background: #c8e0bf; ">

Now, let's try to get a sense for how this changes with the number of neurons we use to train the classifier. 
    
How many neurons do you need to decode with roughly 50% accuracy? 80%? 90%? Finish the code below.
    
For speed reasons, we again suggest using the KFold cross-validation strategy here.

In [ ]:
pop_sizes = np.arange(1, num_units+5, 5).astype('int')
num_resamples = 10
accuracies = np.zeros((len(pop_sizes), num_resamples, num_splits))


# With these analyses in hand, we leave you with some questions:

### If you integrate spikes in a fixed window length, how does the decoding accuracy depend on the time since the image presentation? 

### Where do the lick time distributions fall on the decoding accuracy vs time curve?

### Is the mouse's hit rate different for familiar or novel change images? Is the change decoding accuracy curve different for familiar vs novel change images?


### Are the accuracy curves different in active vs passive blocks?

### Are other variables, including behavioral variables, also encoded in the population activity? Can you decode the running speed, pupil diameter, or licking behavior?

### What about in a different brain area? For example, is the change encoded in CA1 activity? What about in the joint activity across brain areas?